<a href="https://colab.research.google.com/github/mabittar/Portfolio/blob/master/YouTube_Scaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraper 

Nesse web scraper  veremos como criar um algorítmo para baixar os metadados dos vídeos publicados em um canal do Youtube utilizando o [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

Esse é apenas um exemplo de aplicação de um Crawler, que pode ser utilizado para registrar preços de um produto, oportunidades de imóveis para venda ou locação, disponibilidade de servidores e assim por diante.

Web Crawler, bot ou web spider é um algoritmo utilizado para encontrar, ler e gravar informações de páginas da internet. É como um robô que varre o caminho indicado e captura as informações que encontra pela frente.

Um dos maiores exemplos de um web crawler é o próprio google. Antes do site estar disponível para pesquisa, um robô lê o web site e cataloga as informações de forma a serem recuperadas numa busca futura.


Para esse web scrapper dividi o script em funções específicas a fim de facilitar qualquer possível manutenção.

## Connect to Target

Gosto muito dos vídeos publicados no canal [Programação Dinâmica](https://www.youtube.com/c/Programa%C3%A7%C3%A3oDin%C3%A2mica/videos), sempre quando tenho alguma dúvida específica procuro o canal deles. Tenho aprendido muito com o Hallison e a Kizzy.

Resolvi fazer esse scraper como uma forma de fixar o aprendizado e explorar o pouco mais da biblioteca Selenium.

### Iniciando o Selenium no Colab

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.implicitly_wait(10)

In [102]:
# importando demais bibliotecas necessárias
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests


In [109]:
# conectando ao site
link = 'https://www.youtube.com/c/Programa%C3%A7%C3%A3oDin%C3%A2mica/videos'

In [108]:
def get_data(link):

  """
  essa função mapeia o sitemap e retorna as subpáginas onde as imagens estão listadas
  entrada: link para o sitemap
  saída: lista de páginas onde as imagens são listadas
  """
  response = requests.get(link)
  if response.status_code != 200:
    raise RuntimeError(f'Houve um erro de conexão. Cód: {response.status_code}')

  # conectando ao site
  wd.get(link)

  #navegando até o final da página
  for _ in range(56):
    wd.find_element_by_tag_name('body').send_keys(Keys.END)

  #obtendo dados do site:
  html = wd.page_source
  soup = BeautifulSoup(html, 'html.parser')
  videos = soup.find_all('div', {"id": "dismissable"})

  return videos

In [110]:
videos = get_data(link)

In [111]:
# checado a quantidade de itens armazenados
len(videos)

242

## Obtendo metadados do canal

In [71]:
# obtendo metadados

meta_list = []

for video in videos:
  meta_d = {}
  meta_d['title'] = video.find('a', {'id': 'video-title'}).text
  meta_d['link'] = "https://www.youtube.com/" + video.find('a', {'id': 'video-title'})['href']
  meta = video.find('div', {'id': 'metadata-line'}).find_all('span')
  meta_d['vis'] = meta[0].text
  meta_d['publ'] = meta[1].text
  meta_list.append(meta_d)
  

In [72]:
# Convertendo para um Pandas DataFrame
df = pd.DataFrame(meta_list)
df

,title,link,vis,publ
0,NÃO É SÓ O WHATSAPP? SUA PRIVACIDADE ESTÁ EM J...,https://www.youtube.com//watch?v=_QFSjhUaJZ8,7.3K views,1 day ago
1,ADEUS WHATSAPP? ENTENDA OS NOVOS TERMOS DE PRI...,https://www.youtube.com//watch?v=qAUbkKtqqZA,38K views,3 days ago
2,"O Que é Árvore AVL, Propriedades da Árvore AVL...",https://www.youtube.com//watch?v=l8IBdCb2BWA,664 views,6 days ago
3,3 LIÇÕES que APRENDI ESTUDANDO para o IME | AP...,https://www.youtube.com//watch?v=s7lO_jHgCS0,965 views,1 week ago
4,ERROS QUE ELIMINAM SUAS CHANCES NO PROCESSO SE...,https://www.youtube.com//watch?v=_FwmXWDh8O0,1.8K views,1 week ago
...,...,...,...,...
237,O QUE É e como se utiliza UMA VARIÁVEL? | Pyth...,https://www.youtube.com//watch?v=tZ-3EGw5IPw,1K views,3 years ago
238,Comparando Números,https://www.youtube.com//watch?v=pB0UeU7SIo0,1.1K views,3 years ago
239,Operando com Números em Python,https://www.youtube.com//watch?v=hysCU5CWl80,1.8K views,3 years ago
240,Instalando Python,https://www.youtube.com//watch?v=_RadDQpPGM0,2K views,3 years ago


## Tratando dados

In [73]:
def converter_vis(df):
  if 'K' in df['vis']:
    k_views = float(df['vis'].split('K')[0])*1000
    return k_views
  elif 'M' in df['vis']:
    m_views = float(df['vis'].split('M')[0])*1000000
    return m_views
  else:
    views = float(df['vis'].split(' ')[0])
    return views

In [74]:
# Convertendo a visualização para inteiros
df['vis_conv'] = df.apply(converter_vis, axis=1)
df['vis_conv'] = df['vis_conv'].astype(int)

In [75]:
df.head()

,title,link,vis,publ,vis_conv
0,NÃO É SÓ O WHATSAPP? SUA PRIVACIDADE ESTÁ EM J...,https://www.youtube.com//watch?v=_QFSjhUaJZ8,7.3K views,1 day ago,7300
1,ADEUS WHATSAPP? ENTENDA OS NOVOS TERMOS DE PRI...,https://www.youtube.com//watch?v=qAUbkKtqqZA,38K views,3 days ago,38000
2,"O Que é Árvore AVL, Propriedades da Árvore AVL...",https://www.youtube.com//watch?v=l8IBdCb2BWA,664 views,6 days ago,664
3,3 LIÇÕES que APRENDI ESTUDANDO para o IME | AP...,https://www.youtube.com//watch?v=s7lO_jHgCS0,965 views,1 week ago,965
4,ERROS QUE ELIMINAM SUAS CHANCES NO PROCESSO SE...,https://www.youtube.com//watch?v=_FwmXWDh8O0,1.8K views,1 week ago,1800


## Alguns Insights

In [119]:
print('O total de publicações no canal são {} vídeos.' .format(len(df)))

O total de publicações no canal são 242 vídeos.


In [118]:
# podemos obter o total de visualizações do canal:
out_sum = df['vis_conv'].sum()
print('A soma total de visualizações é {:,}.' .format(out_sum))

A soma total de visualizações é 1,078,582.
